In [ ]:
#list all files from bulk:
import dxpy
import os
import subprocess
import zipfile
import shutil

import pandas as pd


In [ ]:
#Download_path
# Step 1: Get the current working directory
current_directory = os.getcwd()
print(f"Current directory: {current_directory}")

# Step 2: Define the name of the new folder
folder_name = "File_DownLoad"  # Change this to the name you want

saving_path = current_directory+'/'+folder_name

if not os.path.exists(folder_name):
    os.mkdir(folder_name)  # Create the directory
    print(f"Folder created: {saving_path}")
else:
    print(f"Folder already exists: {saving_path}")
    
    
# Step 3: Cloud goal 
phenotype_origin_folder = "/GWAS_pipeline"

In [ ]:
# Run the command and capture the output
FREESURFER_HOME = subprocess.run('echo $FREESURFER_HOME', shell=True, check=True, capture_output=True, text=True)

# Get the stdout and strip any extra whitespace or newlines
freesurfer_path = FREESURFER_HOME.stdout.strip()

# Print the result
print(f"Freesurfer path is: {freesurfer_path}")

#copy the template to saving path
copy_command = f'cp -a {freesurfer_path}/subjects/fsaverage {saving_path}'
subprocess.run(copy_command, shell=True, check=True)

#download annotate files
# project_id = dxpy.api.user_get_project({'project': dxpy.DXProject('').get_id()})['project']
# project_name = dxpy.DXProject(project_id).describe()['name']
project_id = 'project-GxqpVq0Jpp5Py82xVbZV198y'  # Replace with the actual project ID


command = f'dx download "{project_id}:/{phenotype_origin_folder}/Freesurfer_related/mask_lh.annot" -o {saving_path}'
subprocess.run(command, shell=True, check=True)
command = f'dx download "{project_id}:/{phenotype_origin_folder}/Freesurfer_related/mask_rh.annot" -o {saving_path}'
subprocess.run(command, shell=True, check=True)
command = f'dx download "{project_id}:/{phenotype_origin_folder}/Freesurfer_related/license.txt" -o {saving_path}'
subprocess.run(command, shell=True, check=True)

#Download list
#load the existed check point:
download_command_check = f'dx download "{project_id}:/Test/subject_list_annot_existed.csv" -o {saving_path}'  
subprocess.run(download_command_check,shell=True, check=True)

# Load subj list
# df_exist = pd.read_csv(saving_path+'/subject_list_annot_existed.csv') 

#copy the licence to freesurfer path for usage
copy_command = f'cp {saving_path}/license.txt {freesurfer_path}'
subprocess.run(copy_command, shell=True, check=True)


In [ ]:
# Environment setting

os.environ['FREESURFER_HOME'] = freesurfer_path
os.environ['SUBJECTS_DIR'] = saving_path+'/'

print(f"freesurfer path exists: {freesurfer_path}")
print(f"subject path exists: {saving_path}")

# Source the setting up
subprocess.run(freesurfer_path+'/sources.sh',shell=True, check=True)



In [ ]:
def surf_transform(file_name,subj_path,freesurfer_path):

    # template
    sfile_annot_lh = subj_path+'/mask_lh.annot'
    tfile_annot_lh = subj_path+'/'+file_name+'_lh.annot'
    
    sfile_annot_rh = subj_path+'/mask_rh.annot'
    tfile_annot_rh = subj_path+'/'+file_name+'_rh.annot'
    
    #################
    command_annot2annot_lh = [freesurfer_path+'/bin/'+'mri_surf2surf',
                        '--srcsubject','fsaverage',
                        '--trgsubject',file_name,
                        '--sval-annot',sfile_annot_lh,
                        '--trgsurfval',tfile_annot_lh,
                        '--hemi','lh']
    
    print(f"surf transform in lh: {command_annot2annot_lh}")
    # subprocess.run(command_annot2annot_lh, shell=True)
    # try:
    #     result = subprocess.run(command_annot2annot_lh, shell=True, capture_output=True, text=True)
    #     print("Command executed successfully")
    #     print("Output:", result.stdout)
    # except subprocess.CalledProcessError as e:
    #     print("Command failed with exit status", e.returncode)
    #     print("Error output:", e.stderr)
    
    command_annot2annot_rh = [freesurfer_path+'/bin/'+'mri_surf2surf',
                        '--srcsubject','fsaverage',
                        '--trgsubject',file_name,
                        '--sval-annot',sfile_annot_rh,
                        '--trgsurfval',tfile_annot_rh,
                        '--hemi','rh']
    
    print(f"surf transform in rh: {command_annot2annot_rh}")
    
    
    return command_annot2annot_lh, command_annot2annot_rh
    
    # subprocess.run(command_annot2annot_rh, shell=True)
    # try:
    #     result = subprocess.run(command_annot2annot_rh, shell=True, capture_output=True, text=True)
    #     print("Command executed successfully")
    #     print("Output:", result.stdout)
    # except subprocess.CalledProcessError as e:
    #     print("Command failed with exit status", e.returncode)
    #     print("Error output:", e.stderr)

In [ ]:
# Define the project ID
# project_id = dxpy.api.user_get_project({'project': dxpy.DXProject('').get_id()})['project']
# project_name = dxpy.DXProject(project_id).describe()['name']
project_id = 'project-GxqpVq0Jpp5Py82xVbZV198y'  # Replace with the actual project ID

# Create a DXProject object for the project
project = dxpy.DXProject(project_id)

# List the contents of the root folder
folder_path = '/Bulk/Brain MRI/T1/'
folder_contents = project.list_folder(folder_path)

#########################
#Test params
Num_file = 0

# df_subj = []#columns = ['folder', 'Name','type','version','prefix']


#########################

for item in folder_contents['folders']:#for each folder
    #List folders
    # print(item)
    folder_name = item.split('/')
    
    # List files
    files = dxpy.find_data_objects(classname='file', folder=item, project=project_id)
    # print(files)
    
    for file in files:#for each file
        ### Check
        # print(file)
        # print(file["id"])
                
        dx_file = dxpy.DXFile(file["id"])
        file_metadata = dx_file.describe()
        # print(f"File Name: {file_metadata['name']}")
        # print(f"File Size: {file_metadata['size']} bytes")
        # print(f"File ID: {file_metadata['id']}")
                
        if '20263' in file_metadata['name']:#According to the file-name, only extract freesurfer files
            # print(f"File Name: {file_metadata['name']}")
            # print(f"File Size: {file_metadata['size']} bytes")
            # print(f"File ID: {file_metadata['id']}")
            
            # Download the files
            Num_file = Num_file+1
            
            # print("subject "+file_metadata['name']+" trial "+str(Num_file)+" is running")
            
#             #######################################
#             #Making the subject csv
#             name_list = file_metadata["name"][:-4].split('_')
            
#             df_subj.append( [folder_name[-1], name_list[0], name_list[1], name_list[2], name_list[3]] )
            
#             # df_subj = df_subj.append({'folder': folder_name[-1], 'Name': name_list[0], 'type': name_list[1],
#             #                'version': name_list[2], 'prefix': name_list[3]}, ignore_index=True)
            
#             #######################################
            
            
            ##### examples：load the existed check point
                
            #######################################
            #Making the subject csv
            name_list = file_metadata["name"][:-4].split('_')
            
            #give the list
            # df_subj.append( [folder_name[-1], name_list[0], name_list[1], name_list[2], name_list[3]] )

            # df_subj = df_subj.append({'folder': folder_name[-1], 'Name': name_list[0], 'type': name_list[1],
            #                'version': name_list[2], 'prefix': name_list[3]}, ignore_index=True)

            #######################################

            
            print('new_data_'+file_metadata["name"][:-4])
        
            # Construct the download command
            print( project_id+":"+item+'/'+file_metadata['id'] )
            print(f"File Name: {file_metadata['name']}")

            download_command = f'dx download "{project_id}:{item}/{file_metadata["id"]}" -o {saving_path}'
            print(download_command)
            subprocess.run(download_command, shell=True, check=True)

            with zipfile.ZipFile(f"{saving_path}/{file_metadata['name']}", 'r') as zip_ref:
                zip_ref.extractall(saving_path)

            # Rename the folder
            os.rename(saving_path+'/FreeSurfer', saving_path+'/'+file_metadata['name'][:-4])


            # template annotate generation
            command_annot2annot_lh, command_annot2annot_rh = surf_transform(file_metadata['name'][:-4],saving_path,freesurfer_path)
            subprocess.run(command_annot2annot_lh)
            subprocess.run(command_annot2annot_rh)
            
            


            # Delete zip and folder file
            shutil.rmtree(saving_path+'/'+file_metadata['name'][:-4])
            # print(f"'{file_metadata['name'][:-4]}' folder has been deleted.")


            ## Download files to Server
            command = f'dx upload {saving_path}/{file_metadata["name"][:-4]}_lh.annot -o {project_id}:/{phenotype_origin_folder}/Annotate_files/{folder_name[-1]}_{file_metadata["name"][:-4]}_lh.annot'
            subprocess.run(command, shell=True, check=True)

            command = f'dx upload {saving_path}/{file_metadata["name"][:-4]}_rh.annot -o {project_id}:/{phenotype_origin_folder}/Annotate_files/{folder_name[-1]}_{file_metadata["name"][:-4]}_rh.annot'
            subprocess.run(command, shell=True, check=True)

            # Remove zip files
            os.remove(saving_path+'/'+file_metadata["name"])
            # print(f"'{file_metadata['name']}' file has been deleted.")

            # Remove annot files
            os.remove(saving_path+'/'+file_metadata["name"][:-4]+'_lh.annot')
            os.remove(saving_path+'/'+file_metadata["name"][:-4]+'_rh.annot')
            # print(f"'{file_metadata['name'][:-4]}' lh.annot and rh.annot has been deleted.")
            

                
